# EdgeNeXt

In [ ]:
!pip install torch torchvision timm

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("elmadafri/the-wildfire-dataset/versions/1")

print("Path to dataset files:", path)

In [ ]:
!rm /root/.cache/kagglehub/datasets/elmadafri/the-wildfire-dataset/versions/1/the_wildfire_dataset/the_wildfire_dataset/val/fire/Both_smoke_and_fire/desktop.ini

In [ ]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from timm import create_model

In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset

class CustomFireDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.binary_labels = []
        self.multi_class_labels = []

        # Define mappings for binary and multi-class labels
        binary_label_mapping = {'fire': 1, 'nofire': 0}
        multi_class_mapping = {
            'fire': {'Both_smoke_and_fire': 0, 'Smoke_from_fires': 1},
            'nofire': {'Fire_confounding_elements': 2, 'Forested_areas_without_confounding_elements': 3, 'Smoke_confounding_elements': 4}
        }

        # Traverse the root directory and collect image paths and labels
        for binary_label_name in os.listdir(root_dir):
            binary_label_path = os.path.join(root_dir, binary_label_name)
            if os.path.isdir(binary_label_path):
                # Assign binary label
                binary_label = binary_label_mapping[binary_label_name]

                # Traverse subclasses
                for subclass_name in os.listdir(binary_label_path):
                    subclass_path = os.path.join(binary_label_path, subclass_name)
                    if os.path.isdir(subclass_path):
                        # Assign multi-class label
                        multi_class_label = multi_class_mapping[binary_label_name][subclass_name]

                        # Collect all images in the subclass directory
                        for img_name in os.listdir(subclass_path):
                            img_path = os.path.join(subclass_path, img_name)
                            if os.path.isfile(img_path):
                                self.image_paths.append(img_path)
                                self.binary_labels.append(binary_label)
                                self.multi_class_labels.append(multi_class_label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        binary_label = self.binary_labels[idx]
        multi_class_label = self.multi_class_labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, (torch.tensor(binary_label, dtype=torch.float), torch.tensor(multi_class_label, dtype=torch.long))


In [ ]:
# Define transformations for the test set
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load your test dataset
test_dataset = CustomFireDataset(root_dir="/root/.cache/kagglehub/datasets/elmadafri/the-wildfire-dataset/versions/1/the_wildfire_dataset/the_wildfire_dataset/test", transform=test_transforms)

# Create a DataLoader for the test dataset
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=10)

## Load Model

In [ ]:
import torch
from torch import nn
from timm import create_model

# Load the pretrained EdgeNeXt model
model = create_model('edgenext_small', pretrained=True)

# Dynamically compute the number of features from forward_features
dummy_input = torch.randn(1, 3, 224, 224)
with torch.no_grad():
    features = model.forward_features(dummy_input)
flattened_features_size = features.reshape(features.size(0), -1).size(1)

# Add two separate heads for binary and multi-class classification
# Binary classification head: Includes Dropout and LeakyReLU
model.head_binary = nn.Sequential(
    nn.Linear(flattened_features_size, flattened_features_size // 2),
    nn.LeakyReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(flattened_features_size // 2, 1)
)

# Multi-class classification head: Includes LeakyReLU
model.head_multiclass = nn.Sequential(
    nn.Linear(flattened_features_size, flattened_features_size // 2),
    nn.LeakyReLU(),
    nn.Linear(flattened_features_size // 2, 5)
)

# Define a custom forward method
def forward_with_two_heads(self, x):
    features = self.forward_features(x)  # Extract features from the backbone
    features = features.reshape(features.size(0), -1)  # Flatten features dynamically using reshape
    binary_output = self.head_binary(features)  # Binary classification
    multiclass_output = self.head_multiclass(features)  # Multi-class classification
    return binary_output, multiclass_output

# Attach the custom forward method to the model
model.forward = forward_with_two_heads.__get__(model, type(model))

In [ ]:
print(model)

In [ ]:
model.load_state_dict(torch.load("edgenext_aug1.pth"))

In [ ]:
import torch.nn as nn
from torch.quantization import quantize_dynamic

model.eval()

quantized_model = quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)


## Test the Model

In [ ]:
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report
import pandas as pd
import numpy as np
import torch

# Ensure the quantized model is on CPU
device = torch.device('cpu')
quantized_model = quantized_model.to(device)
quantized_model.eval()

# Initialize lists to store results
true_binary_labels = []
predicted_binary_probs = []  # Probabilities for the positive class (fire)

true_multi_class_labels = []
predicted_multi_class_labels = []

with torch.no_grad():
    for images, (binary_labels, multi_class_labels) in test_loader:
        # Move data to CPU (this is often redundant if dataloader is already on CPU, but it's safe)
        images = images.to(device)
        binary_labels = binary_labels.to(device)
        multi_class_labels = multi_class_labels.to(device)

        # Forward pass through the quantized model
        binary_output, multi_class_output = quantized_model(images)

        # Convert binary logits to probabilities
        binary_probs = torch.sigmoid(binary_output).squeeze().cpu().numpy()

        # Get predicted class labels for multi-class classification
        _, predicted_multi_class = torch.max(multi_class_output, 1)
        predicted_multi_class = predicted_multi_class.cpu().numpy()

        # Store true labels and predictions for binary classification
        true_binary_labels.extend(binary_labels.cpu().numpy())
        predicted_binary_probs.extend(binary_probs)

        # Store true labels and predictions for multi-class classification
        true_multi_class_labels.extend(multi_class_labels.cpu().numpy())
        predicted_multi_class_labels.extend(predicted_multi_class)

# Combine all data into a single DataFrame
results = pd.DataFrame({
    "True Binary Labels": true_binary_labels,
    "Predicted Binary Probabilities": predicted_binary_probs,
    "True Multi-Class Labels": true_multi_class_labels,
    "Predicted Multi-Class Labels": predicted_multi_class_labels
})

# Save results to CSV
results.to_csv("edgenext_aug1_test_results.csv", index=False)
print("Test results saved to 'edgenext_aug1_test_results.csv'.")

In [ ]:
torch.save(quantized_model.state_dict(), "edgenext_quantized_model.pth")

In [ ]:
import time  # For measuring time
import pandas as pd  # For saving to CSV
import torch

# Ensure quantized_model is on CPU and in eval mode
device = torch.device('cpu')
model.to(device)
model.eval()

# List to store prediction times
prediction_times = []

with torch.no_grad():
    for images, _ in test_loader:  # Only using images; ignoring labels
        images = images.to(device)

        # Start timing the prediction
        start_time = time.time()

        # Forward pass through the quantized model directly
        binary_output, multi_class_output = model(images)

        # End timing the prediction
        end_time = time.time()

        # Calculate and store the time taken for this prediction
        prediction_time = end_time - start_time
        prediction_times.append(prediction_time)

# Save the prediction times to a CSV file
data = pd.DataFrame({"Prediction Times (seconds)": prediction_times})
data.to_csv("cpu_edgenext_prediction_times.csv", index=False)

In [ ]:
import time  # For measuring time
import pandas as pd  # For saving to CSV
import torch

# Ensure quantized_model is on CPU and in eval mode
device = torch.device('cpu')
quantized_model = quantized_model.to(device)
quantized_model.eval()

# List to store prediction times
prediction_times = []

with torch.no_grad():
    for images, _ in test_loader:  # Only using images; ignoring labels
        images = images.to(device)

        # Start timing the prediction
        start_time = time.time()

        # Forward pass through the quantized model directly
        binary_output, multi_class_output = quantized_model(images)

        # End timing the prediction
        end_time = time.time()

        # Calculate and store the time taken for this prediction
        prediction_time = end_time - start_time
        prediction_times.append(prediction_time)

# Save the prediction times to a CSV file
data = pd.DataFrame({"Prediction Times (seconds)": prediction_times})
data.to_csv("quantized_edgenext_prediction_times.csv", index=False)